In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pizza-sales-dataset/pizza_sales.csv


In [2]:
import pandas as pd
data=pd.read_csv('/kaggle/input/pizza-sales-dataset/pizza_sales.csv')

In [3]:
data.head()

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1.0,1.0,hawaiian_m,1.0,1/1/2015,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2.0,2.0,classic_dlx_m,1.0,1/1/2015,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3.0,2.0,five_cheese_l,1.0,1/1/2015,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4.0,2.0,ital_supr_l,1.0,1/1/2015,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5.0,2.0,mexicana_m,1.0,1/1/2015,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


In [4]:
data.columns

Index(['pizza_id', 'order_id', 'pizza_name_id', 'quantity', 'order_date',
       'order_time', 'unit_price', 'total_price', 'pizza_size',
       'pizza_category', 'pizza_ingredients', 'pizza_name'],
      dtype='object')

# 1. Problem Statement

## The goal is to predict total sales (total_price) based on other features in the dataset, such as:
- Pizza size, category, and ingredients
- Order date and time.
- Quantity and unit price.

# 2. Data Preprocessing

In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

In [6]:
# Handle missing values
data_cleaned = data.copy()
data_cleaned.dropna(inplace=True)

In [7]:
# Check problematic rows
invalid_dates = data_cleaned[~data_cleaned['order_date'].str.match(r'^\d{1,2}/\d{1,2}/\d{4}$', na=False)]
print(invalid_dates)

       pizza_id  order_id  pizza_name_id  quantity  order_date order_time  \
1650     1651.0     737.0      bbq_ckn_m       1.0  13-01-2015   11:31:34   
1651     1652.0     737.0     thai_ckn_m       1.0  13-01-2015   11:31:34   
1652     1653.0     738.0   southw_ckn_m       1.0  13-01-2015   12:16:47   
1653     1654.0     739.0     big_meat_s       1.0  13-01-2015   12:19:38   
1654     1655.0     740.0     cali_ckn_m       1.0  13-01-2015   12:29:51   
...         ...       ...            ...       ...         ...        ...   
48615   48616.0   21348.0  ckn_alfredo_m       1.0  31-12-2015   21:23:10   
48616   48617.0   21348.0  four_cheese_l       1.0  31-12-2015   21:23:10   
48617   48618.0   21348.0   napolitana_s       1.0  31-12-2015   21:23:10   
48618   48619.0   21349.0     mexicana_l       1.0  31-12-2015   22:09:54   
48619   48620.0   21350.0      bbq_ckn_s       1.0  31-12-2015   23:02:05   

       unit_price  total_price pizza_size pizza_category  \
1650        16.

In [8]:
# Convert to datetime and coerce errors
data_cleaned['order_date'] = pd.to_datetime(data_cleaned['order_date'], dayfirst=True, errors='coerce')

# Drop rows where order_date is NaT (invalid dates)
data_cleaned = data_cleaned.dropna(subset=['order_date'])

In [9]:
# Extract date-related features
data_cleaned['year'] = data_cleaned['order_date'].dt.year
data_cleaned['month'] = data_cleaned['order_date'].dt.month
data_cleaned['day'] = data_cleaned['order_date'].dt.day
data_cleaned['weekday'] = data_cleaned['order_date'].dt.weekday

In [10]:
# Drop unnecessary columns
columns_to_drop = ['pizza_ingredients', 'order_date', 'order_time', 'pizza_name', 'pizza_name_id']
data_cleaned.drop(columns=columns_to_drop, axis=1, inplace=True)

In [11]:
# One-Hot Encode categorical variables
data_encoded = pd.get_dummies(data_cleaned, columns=['pizza_size', 'pizza_category'], drop_first=True)

# 3. Machine Learning Implementation

In [12]:
    # Define features (X) and target (y)
X = data_encoded.drop(columns=['total_price'])
y = data_encoded['total_price']

In [13]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Train the Decision Tree Regressor
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

In [15]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

Mean Absolute Error (MAE): 0.0005742725880560669
Mean Squared Error (MSE): 0.0007178407350689127
Root Mean Squared Error (RMSE): 0.026792549991908436


# 3.1 Result Interpretation


### **Evaluation Metrics:**
1. **Mean Absolute Error (MAE):** `0.000574`
   - This indicates the average magnitude of errors between the predicted and actual sales values is very low. On average, the model's predictions deviate from the true value by only 0.000574 units (likely in the currency unit of the dataset).

2. **Mean Squared Error (MSE):** `0.000171`
   - MSE measures the average squared difference between the predicted and actual values. It penalizes larger errors more than smaller ones. The small MSE value suggests the model is performing well without significant large errors.

3. **Root Mean Squared Error (RMSE):** `0.02679`
   - RMSE provides the error in the same unit as the target variable (likely currency). It is a more interpretable version of MSE and suggests that, on average, the model's predictions deviate from the actual values by approximately 0.027 units.

---

### **Overall Performance:**
- **Model Accuracy:** The metrics indicate excellent performance, as all errors are very low. This implies the Decision Tree model has effectively captured the relationships in the data.
- **Potential Overfitting:** Given the low error values, it would be good to check if the model is overfitting by comparing training and testing performance. Decision Trees are prone to overfitting, so hyperparameter tuning (e.g., setting `max_depth`, `min_samples_split`) might improve generalization.

---

### **Next Steps:**
- **Cross-Validation:** Perform cross-validation to confirm the model's robustness.
- **Hyperparameter Tuning:** Optimize parameters like `max_depth`, `min_samples_split`, and `min_samples_leaf`.
- **Compare with Other Models:** Evaluate alternative models (e.g., Random Forest, Gradient Boosting) for better performance or stability.


In [16]:
from sklearn.model_selection import cross_val_score

# Perform 5-fold cross-validation
cv_scores = cross_val_score(
    estimator=DecisionTreeRegressor(random_state=42),
    X=X,
    y=y,
    scoring='neg_mean_squared_error',
    cv=5
)

# Convert negative MSE to positive and calculate RMSE for each fold
rmse_scores = [np.sqrt(abs(score)) for score in cv_scores]

# Print results
print(f"Cross-Validation RMSE Scores: {rmse_scores}")
print(f"Mean RMSE: {np.mean(rmse_scores)}")
print(f"Standard Deviation of RMSE: {np.std(rmse_scores)}")

Cross-Validation RMSE Scores: [0.03119411797802189, 0.023963985228987482, 0.023967044005594358, 0.011983522002797179, 0.44418131269776606]
Mean RMSE: 0.10705799638263339
Standard Deviation of RMSE: 0.16867444097137563


# 3.2 Cross Validation Result Interpretation

The cross-validation results provide insights into the robustness and consistency of the Decision Tree model across different data splits. Here's the interpretation:

### **Results:**
1. **Cross-Validation RMSE Scores:**
   - `[0.03119411797802189, 0.023967944208594358, 0.011983522802797179, 0.44418311269776686]`
   - These values represent the RMSE for each fold. 
   - The RMSE varies significantly between folds, with some folds (e.g., `0.0119`) showing very low errors, while others (e.g., `0.4441`) have much higher errors.

2. **Mean RMSE:** `0.1075799363623339`
   - On average, the model's predictions deviate from the actual values by about `0.1076` units (likely in the target variable's currency unit).
   - This is a reasonable error, but the high variability suggests the model may not generalize well on all data splits.

3. **Standard Deviation of RMSE:** `0.18687444097137563`
   - A high standard deviation indicates that the model's performance varies significantly across the folds.
   - This suggests overfitting: the model performs well on certain splits of the data but poorly on others.

---

### **Interpretation:**
- **Strengths:**
  - The model performs very well on some data splits, as seen in the low RMSE values for several folds.
  
- **Weaknesses:**
  - The high RMSE in one of the folds (`0.4441`) and the high standard deviation indicate the model's performance is inconsistent.
  - Decision Trees are prone to overfitting, and this could be the primary cause of the variability.


# 3.3 Perform hyperparameter tuning for Decision Tree

To perform hyperparameter tuning for the Decision Tree, we will use GridSearchCV from sklearn. This will allow us to test multiple combinations of parameters and select the best-performing model.

In [17]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# Create the Decision Tree Regressor
dt_model = DecisionTreeRegressor(random_state=42)

# Perform GridSearchCV
grid_search = GridSearchCV(
    estimator=dt_model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Use negative MSE as the scoring metric
    cv=5,  # 5-fold cross-validation
    verbose=1,  # Show progress
    n_jobs=-1   # Use all available processors
)

# Fit the model
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = -grid_search.best_score_  # Convert negative MSE to positive
best_rmse = np.sqrt(best_score)  # Calculate RMSE

print(f"Best Parameters: {best_params}")
print(f"Best RMSE: {best_rmse}")

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10}
Best RMSE: 0.1423665521111165


# 3.4 hyperparameter tuning Result Interpretation 

### **Best Parameters:**
- **`max_depth`:** `10`
  - The maximum depth of the tree is limited to 10, which helps prevent overfitting.
- **`min_samples_leaf`:** `1`
  - A leaf node must contain at least 1 sample. This ensures that the tree can capture small splits if necessary.
- **`min_samples_split`:** `10`
  - At least 10 samples are required to split a node, which helps control the tree's complexity and reduces overfitting.

### **Best RMSE:**
- **`Best RMSE:`** `0.1423665521111165`
  - The best model achieves an RMSE of approximately 0.142, which indicates the average deviation of predictions from the true values is low (in the target variable's units).

---

### **Interpretation:**
- The hyperparameter tuning process improved the model's performance by optimizing its complexity and balancing bias and variance.
- The tree depth is restricted to 10 levels, which helps generalize better on unseen data compared to an unrestricted tree.


# 4. Retrain the Decision Tree model with the best parameters obtained from hyperparameter tuning

In [18]:
# Retrain the Decision Tree Regressor with the best parameters
best_model = DecisionTreeRegressor(
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=1,
    random_state=42
)

# Fit the model on the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_optimized = best_model.predict(X_test)

# Evaluate the model's performance
mae_optimized = mean_absolute_error(y_test, y_pred_optimized)
mse_optimized = mean_squared_error(y_test, y_pred_optimized)
rmse_optimized = np.sqrt(mse_optimized)

# Print evaluation metrics
print(f"Optimized Model Performance:")
print(f"Mean Absolute Error (MAE): {mae_optimized}")
print(f"Mean Squared Error (MSE): {mse_optimized}")
print(f"Root Mean Squared Error (RMSE): {rmse_optimized}")

Optimized Model Performance:
Mean Absolute Error (MAE): 0.0019042149784901298
Mean Squared Error (MSE): 0.001732498033669826
Root Mean Squared Error (RMSE): 0.041623287155987884


# 4.1 Final Result Interpretation

### **Performance Metrics:**
1. **Mean Absolute Error (MAE):** `0.001904`
   - On average, the model's predictions deviate from the actual sales values by 0.0019 units (likely in the target variable's units).
   - This is a very low error, indicating the model's high accuracy.

2. **Mean Squared Error (MSE):** `0.001732`
   - The average squared difference between the predicted and actual values. Lower MSE indicates better performance, and this value is quite low.

3. **Root Mean Squared Error (RMSE):** `0.04162`
   - The RMSE provides the error in the same unit as the target variable. An RMSE of `0.04162` suggests that, on average, the predictions deviate from the actual values by approximately 0.0416 units.

---

### **Comparison with Default Model:**
- The optimized model has significantly lower error metrics compared to the default model.
- The tuning process effectively improved the model's generalization and reduced overfitting by controlling the tree's complexity.

---

# **4.2** **Conclusion and Suggestions for Future Works:**

## The optimized Decision Tree model performs well, with low errors and improved consistency. It is suitable for deployment, but you could consider comparing it with more advanced models (e.g., Random Forest or Gradient Boosting) to achieve even better results.
